# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686597


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:02<00:41,  1.47s/it]

Rendering models:  10%|█         | 3/30 [00:05<00:49,  1.84s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:46,  1.79s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:32,  1.29s/it]

...

Rendering models:  83%|████████▎ | 25/30 [00:09<00:00,  7.11it/s]

Rendering models:  87%|████████▋ | 26/30 [00:10<00:00,  5.87it/s]

Rendering models:  90%|█████████ | 27/30 [00:10<00:00,  5.43it/s]

Rendering models:  97%|█████████▋| 29/30 [00:10<00:00,  5.73it/s]

not-logged-in-1737c3b79f1a3fbc13ec    0.000554
not-logged-in-7811b1412330729c57cc    0.001488
not-logged-in-3daf853161d1992c7d3e    0.036682
tanya_p                               0.000574
Kiyabelle                             0.003609
austingasbarra                        0.000563
kjkavene                              0.000538
rnevils                               0.000569
justsomenoodles                       0.000564
lsautter                              0.000564
bskjoren                              0.001943
not-logged-in-314d48142a3a122ba666    0.002176
41Letters                             0.000668
Lavadude                              0.002126
equidad1                              0.000548
onustu                                0.000594
ktayl47                               0.000599
not-logged-in-31cd0f5488b385c0cc95    0.007825
jmacd297                              0.001948
syt18                                 0.001478
Getrekt10127                          0.007341
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())